In [0]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git


  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-9dxjk5g_
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-9dxjk5g_
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4307 sha256=e7d07c0f7d630551345ea8344229b4e59708b3ebbd41fd28a633e86fae223d2b
  Stored in directory: /tmp/pip-ephem-wheel-cache-3l9l8_xh/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [0]:
%load_ext nvcc_plugin


The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin


In [0]:
  # C(i, j = (A(i, j) − B(i, j)^3) 
  # print 3 tables

In [0]:
%%cu
#include <iostream>
#include <stdio.h>
using namespace std;

#define COLS 3
#define ROWS 2

__global__ void func(int *d_a, int *d_b, int *d_c){
 
 //index for each thread
 int indx = threadIdx.x + blockIdx.x * blockDim.x;
 
 //calculate -> C(i, j = (A(i, j) − B(i, j)^3) 
 d_c[indx] = pow(d_a[indx] - d_b[indx], 3);

}



int main(void){
    
 //device copies of a, b, c
 int *d_a, *d_b, *d_c;
 
 //size to allocate memory on device
 int size = COLS * ROWS * sizeof(int);

 //initialize 2d arrays: a, b, c, 
 int a[][3] = { {10, 10, 10}, 
                {10, 10, 10} };

 int b[][3] = { {1, 2, 3}, 
                {4, 5, 6} };
   
 int c[2][3] = { {0, 0, 0}, 
                {0, 0, 0} };
 
 //Allocate memory for device's copies
 cudaMalloc((void** )&d_a, size); 
 cudaMalloc((void** )&d_b, size);
 cudaMalloc((void** )&d_c, size);

 //Copy  a, b from host to device
 cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);
 cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);

 //Setup the execution configuration
 dim3 dimBlock(2, 1);
 dim3 threadsPerBlock(6, 1);

 //Launch function on device
 func <<< dimBlock, threadsPerBlock >>>(d_a, d_b, d_c);

 //Copy c from device to host
 cudaMemcpy(c, d_c, size, cudaMemcpyDeviceToHost);
 
 //free, allocated space
 cudaFree(d_a);
 cudaFree(d_b);
 cudaFree(d_c);
 
 //display results
 printf("  C(i, j = (A(i, j) − B(i, j)^3) \n\n");
 for(int i = 0; i < ROWS; ++i){
     
     for(int j = 0; j < COLS; ++j){
         
         
         printf("[%d][%d] \t",i, j);
         printf("c: %d = (a: %d - b: %d)^3  \n", c[i][j], a[i][j], b[i][j] );
         
         }
 } 

 return 0;
}

  C(i, j = (A(i, j) − B(i, j)^3) 

[0][0] 	c: 729 = (a: 10 - b: 1)^3  
[0][1] 	c: 512 = (a: 10 - b: 2)^3  
[0][2] 	c: 343 = (a: 10 - b: 3)^3  
[1][0] 	c: 216 = (a: 10 - b: 4)^3  
[1][1] 	c: 125 = (a: 10 - b: 5)^3  
[1][2] 	c: 64 = (a: 10 - b: 6)^3  

